In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
input_dir = '/Users/jupternguyen/Projects/TrafficFlowPrediction/data/raw_scats/Scats Data October 2006.csv'
output_dir = '/Users/jupternguyen/Projects/TrafficFlowPrediction/data/separated_scats/'
os.makedirs(output_dir, exist_ok=True)

In [3]:
df = pd.read_csv(input_dir)

In [4]:
# Drop the last three columns
df = df.iloc[:, :-3]
print(df.head())

   SCATS Number                         Location CD_MELWAY  NB_LATITUDE  \
0           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
1           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
2           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
3           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
4           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   

   NB_LONGITUDE  HF VicRoads Internal  VR Internal Stat  VR Internal Loc  \
0     145.09159                   249               182                1   
1     145.09159                   249               182                1   
2     145.09159                   249               182                1   
3     145.09159                   249               182                1   
4     145.09159                   249               182                1   

   NB_TYPE_SURVEY        Date  ...  V86  V87  V88  V89  V90  V91  V92  V93  \
0             

In [5]:
# Check for missing values
na_counts = df.isna().sum().sum() # three unrelated columns
print(na_counts)

# Check for duplicates
duplicate_counts = df.duplicated().sum()
print(duplicate_counts)

0
0


In [6]:
# Initialize an empty list to store the transformed data
transformed_data = []

# Loop through each row in the dataset
for idx, record in df.iterrows():
    scats_id = record['SCATS Number']  # Extract SCATS Number
    site_location = record['Location']  # Extract location
    record_date = record['Date']  # Extract date

    # Loop through 96 intervals, representing 15-minute blocks in a day
    for interval in range(96):
        # Append a dictionary with the transformed row to the transformed_data list
        transformed_data.append({
            'SCATS Number': scats_id,
            'Location': site_location,
            '15 Minutes': record_date,  # Append date as part of the '15 Minutes' column
            'Lane 1 Flow (Veh/15 Minutes)': record[f'V{interval:02d}'],  # Vehicle flow data for each 15-minute interval
            '# Lane Points': 1,  # Assumed to be 1 lane point
            '% Observed': 100  # Assumed 100% data observation
        })

print(f'Transformed {len(transformed_data)} records')

Transformed 402432 records


In [7]:
# Create a DataFrame from the transformed data (ignoring the extra columns initially)
df_transformed = pd.DataFrame(transformed_data)

# Format the SCATS Number as a 4-digit string for consistency
df_transformed['SCATS Number'] = df_transformed['SCATS Number'].apply(lambda x: f'{x:04d}')

# Generate time intervals for each 15-minute block in a day
time_blocks = pd.date_range(start='00:00', end='23:45', freq='15min').strftime('%H:%M').tolist()
df_transformed['Time'] = time_blocks * (len(df_transformed) // len(time_blocks))
df_transformed['15 Minutes'] = df_transformed['15 Minutes'] + ' ' + df_transformed['Time']
df_transformed = df_transformed.drop(columns=['Time'])

# Convert the '15 Minutes' column to datetime (adjust format as needed)
df_transformed['15 Minutes'] = pd.to_datetime(df_transformed['15 Minutes'], format='%d-%m-%Y %H:%M')

# Create new features based on the datetime
df_transformed['Day'] = df_transformed['15 Minutes'].dt.dayofweek + 1
df_transformed['Hour'] = df_transformed['15 Minutes'].dt.hour + df_transformed['15 Minutes'].dt.minute / 60.0
df_transformed['Hour-Sin'] = np.sin(2 * np.pi * df_transformed['Hour'] / 24)
df_transformed['Hour-Cos'] = np.cos(2 * np.pi * df_transformed['Hour'] / 24)

# Create lag features for the flow column grouped by SCATS Number
df_transformed['Lag-15min'] = df_transformed.groupby('SCATS Number')['Lane 1 Flow (Veh/15 Minutes)'].shift(1)
df_transformed['Lag-30min'] = df_transformed.groupby('SCATS Number')['Lane 1 Flow (Veh/15 Minutes)'].shift(2)
df_transformed['Lag-15min'] = df_transformed['Lag-15min'].fillna(0)
df_transformed['Lag-30min'] = df_transformed['Lag-30min'].fillna(0)

# Rearrange columns to match the desired order
ordered_columns = ['SCATS Number', 'Location', '15 Minutes', 'Day', 'Hour', 'Hour-Sin', 'Hour-Cos',
                   'Lag-15min', 'Lag-30min', 'Lane 1 Flow (Veh/15 Minutes)', '# Lane Points', '% Observed']
df_transformed = df_transformed[ordered_columns]

# Group the transformed data by 'Location'
location_groups = df_transformed.groupby('Location')

print(f'Found {len(location_groups)} unique locations')

Found 139 unique locations


In [8]:
# Save each location's data to a separate CSV file
for site_location, group_data in location_groups:
    # Construct the file name based on SCATS number and location
    output_file_name = f'{group_data["SCATS Number"].iloc[0]}_{site_location}.csv'
    output_file_path = os.path.join(output_dir, output_file_name)
    # Save the grouped data to a CSV file
    group_data.to_csv(output_file_path, index=False)